In [ ]:
import sys
if "google.colab" in sys.modules:
    !wget "https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py"
    import helper
    helper.install_glpk ()
    helper.install_idaes()
    helper.install_ipopt()
!pip install requests folium
!pip install openrouteservice

In [36]:
import pandas as pd

df_clients = pd.read_csv('./content/caso3/clients.csv')
df_depots = pd.read_csv('./content/caso3/depots.csv')
df_vehicles = pd.read_csv('./content/caso3/vehicles.csv')

In [37]:
df_depots['DepotID'] = df_depots['DepotID'].astype(str)
df_depots['ClientID'] = df_clients['ClientID'].astype(str)
df_vehicles['VehicleName'] = [f'V{i+1}' for i in range(len(df_vehicles))]


coord_depots = list(zip(df_depots['Longitude'], df_depots['Latitude']))

coord_clients = list(zip(df_clients['Longitude'], df_clients['Latitude']))

lugares = coord_depots + coord_clients

In [38]:
from pyomo.environ import *
import requests

model = ConcreteModel()

# Definición de conjuntos
model.D = Set(initialize=df_depots['DepotID'].apply(lambda x: f'CD{x}'))
model.C = Set(initialize=df_clients['ClientID'].apply(lambda x: f'C{x}'))
model.V = Set(initialize=df_vehicles['VehicleName'].tolist())
model.N = model.D | model.C


print("model.D =", list(model.D.data()))
print("model.C =", list(model.C.data()))
print("model.V =", list(model.V.data()))
print("model.N =", list(model.N.data()))
print((lugares))
print(len(lugares))

model.D = ['CD1', 'CD2', 'CD3', 'CD4', 'CD5', 'CD6', 'CD7', 'CD8', 'CD9', 'CD10', 'CD11', 'CD12']
model.C = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29', 'C30', 'C31', 'C32', 'C33', 'C34', 'C35', 'C36', 'C37', 'C38', 'C39', 'C40', 'C41', 'C42', 'C43', 'C44', 'C45', 'C46', 'C47', 'C48', 'C49', 'C50', 'C51', 'C52', 'C53', 'C54', 'C55', 'C56', 'C57', 'C58', 'C59', 'C60', 'C61', 'C62', 'C63', 'C64', 'C65', 'C66', 'C67', 'C68', 'C69', 'C70', 'C71', 'C72', 'C73', 'C74', 'C75', 'C76', 'C77', 'C78', 'C79', 'C80', 'C81', 'C82', 'C83', 'C84', 'C85', 'C86', 'C87', 'C88', 'C89', 'C90']
model.V = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30']
model.N = ['CD1', 'CD2', 'CD3', 'CD4', 'CD5', 'CD6', 'CD7'

In [39]:
lugares[:50]

[(-74.08124218159384, 4.75021190869025),
 (-74.10993358606953, 4.5363832206427785),
 (-74.03854814565923, 4.792925960208614),
 (-74.06706883098641, 4.72167778077445),
 (-74.13826337931849, 4.607707046760958),
 (-74.12400186370824, 4.650463053612691),
 (-74.09561875464892, 4.621911772492814),
 (-74.10975623736951, 4.678960680833056),
 (-74.09547235719887, 4.735973062153282),
 (-74.10991610076434, 4.550640992537941),
 (-74.10977422186991, 4.664702960902753),
 (-74.12408925943333, 4.5791740634103135),
 (-74.15080587752655, 4.679769454542633),
 (-74.01799606682779, 4.733830991590818),
 (-74.14365528529937, 4.620928444870193),
 (-74.03458280137461, 4.7330740665555),
 (-74.05588712830665, 4.820900231096688),
 (-74.04633863054056, 4.779275206620083),
 (-74.06470768583738, 4.723527744298546),
 (-74.16228381122208, 4.551786739961474),
 (-74.09545310710507, 4.7263261853539165),
 (-74.06437527371203, 4.662353891105346),
 (-74.14583743858148, 4.682508652030651),
 (-74.0435651390896, 4.669647000002

In [ ]:
import openrouteservice
from openrouteservice import convert
import json
import csv

API_KEY = '5b3ce3597851110001cf6248781c341d9b2c47b4b5f6ce22fb428092'

client = openrouteservice.Client(key=API_KEY)
precio_km = 3032.1

matrix = client.distance_matrix(
    locations=lugares[:50],
    profile='driving-car',
    metrics=['distance'],
    resolve_locations=True,
    units='km',
)

print(matrix)

In [41]:
precio_km = 3032.1
cost = {}
distance = {}

dist_matrix = matrix['distances']
print(dist_matrix)
print(len(dist_matrix))

for i,i1 in zip(range(len(lugares)),  list(model.N.data())):
        for j,j1 in zip(range(len(lugares)),  list(model.N.data())):
            if i != j:
                distancia_km = dist_matrix[i][j]
                precio = round(distancia_km * precio_km, 3)
            else:
                distancia_km = 9999999
                precio =9999999
            distance[(i1, j1)] = distancia_km
            cost[(i1, j1)] = precio
print(distance)
print(cost)


[[0.0, 32.99, 10.7, 7.49, 20.95, 17.04, 21.1, 13.71, 4.33, 30.68, 13.63, 26.3, 16.12, 10.47, 19.79, 9.12, 13.17, 12.13, 7.34, 31.32, 5.42, 15.13, 15.39, 17.45, 18.41, 15.86, 22.17, 20.65, 23.26, 23.64, 22.1, 11.75, 8.09, 26.21, 22.95, 18.56, 30.88, 15.34, 26.82, 18.98, 7.19, 18.05, 14.0, 14.55, 22.91, 22.15, 20.26, 28.55, 23.88, 14.33], [33.54, 0.0, 34.08, 29.96, 13.84, 19.16, 14.14, 24.19, 32.07, 2.58, 22.37, 6.42, 24.21, 32.38, 15.67, 29.96, 40.18, 35.51, 30.35, 11.62, 31.03, 19.39, 24.01, 18.82, 20.0, 17.62, 12.12, 14.38, 19.0, 12.96, 19.22, 23.24, 29.51, 11.69, 10.61, 15.14, 4.71, 21.73, 6.38, 16.58, 29.14, 15.9, 39.65, 22.69, 20.83, 12.24, 13.69, 11.7, 9.33, 22.15], [13.9, 38.88, 0.0, 15.76, 32.09, 28.18, 26.98, 24.86, 17.63, 36.57, 24.77, 32.18, 30.19, 10.52, 30.94, 13.79, 6.1, 7.04, 14.77, 41.65, 20.73, 19.84, 29.47, 21.3, 31.99, 20.57, 28.05, 31.8, 34.41, 31.99, 35.68, 18.9, 23.81, 32.09, 28.83, 24.44, 36.76, 29.41, 32.7, 30.13, 20.61, 29.19, 5.57, 28.62, 36.49, 28.03, 24.97, 3

IndexError: list index out of range

In [23]:
depot_capacity = {}
for depot, d in zip(df_depots['DepotID'], df_depots['Capacity']):
    depot_capacity[f'CD{depot}'] = d

print(depot_capacity)

demand = {}
for client, c in zip(df_clients['ClientID'], df_clients['Demand']):
    demand[f'C{client}'] = c

print(demand)

vehicle_capacity={}
for vehicle, c in zip(df_vehicles['VehicleName'], df_vehicles['Capacity']):
    vehicle_capacity[vehicle] = c

print(vehicle_capacity)

vehicle_range={}
for vehicle, r in zip(df_vehicles['VehicleName'], df_vehicles['Range']):
    vehicle_range[vehicle] = r

print(vehicle_range)


{'CD1': 8, 'CD2': 10, 'CD3': 0, 'CD4': 4, 'CD5': 28, 'CD6': 3, 'CD7': 0, 'CD8': 10, 'CD9': 43, 'CD10': 1, 'CD11': 16, 'CD12': 18}
{'C1': 12, 'C2': 15, 'C3': 15, 'C4': 6, 'C5': 5, 'C6': 11, 'C7': 12, 'C8': 10, 'C9': 15}
{'V1': 131.9211396722696, 'V2': 108.4356199315333, 'V3': 91.50425520531196, 'V4': 32.896064077536955, 'V5': 22.65262807032524, 'V6': 22.682911535937688}
{'V1': 145.85207096486445, 'V2': 1304.605971281605, 'V3': 953.172608610164, 'V4': 17.302304187458727, 'V5': 16.627680130757895, 'V6': 13.602810739289229}


In [24]:

model.cost = Param(model.N, model.N, initialize=lambda model, i, j: cost[(i, j)], within=NonNegativeReals)
model.distance = Param(model.N, model.N, initialize=lambda model, i, j: distance[(i, j)], within=NonNegativeReals)

# Variables de decisión y auxiliares

model.x = Var(model.V, model.N, model.N, domain=Binary)
model.y = Var(model.V, model.D, domain=Binary)
model.u = Var(model.V, model.C, domain=NonNegativeReals)

# funcion objetivo

def obj_rule(model):
    return sum(model.cost[i,j] * model.x[v,i,j] for v in model.V for i in model.N for j in model.N if i != j)

model.obj = Objective(rule=obj_rule, sense=minimize)

# Restricciones

def assign_depot_rule(model, v):
    return sum(model.y[v,d] for d in model.D) == 1

model.assign_depot = Constraint(model.V, rule=assign_depot_rule)

def start_route_rule(model, v, d):
    return sum(model.x[v, d, j] for j in model.N if j != d) == model.y[v,d]

model.start_route = Constraint(model.V, model.D, rule=start_route_rule)

def return_route_rule(model, v, d):
    return sum(model.x[v, i, d] for i in model.N if i != d) == model.y[v,d]

model.return_route = Constraint(model.V, model.D, rule=return_route_rule)


def same_depot_entry_exit_rule(model, v, d):
    # Solo se puede entrar a un depósito si se ha salido de él
    return (
        sum(model.x[v, d, j] for j in model.N if j != d)
        ==
        sum(model.x[v, i, d] for i in model.N if i != d)
    )

model.same_depot_entry_exit = Constraint(model.V, model.D, rule=same_depot_entry_exit_rule)

def flow_conservation_rule(model, v, i):
    if i in model.C:
        return sum(model.x[v, j, i] for j in model.N if j != i) == \
               sum(model.x[v, i, j] for j in model.N if j != i)
    else:
        return Constraint.Skip

model.flow_conservation = Constraint(model.V, model.N, rule=flow_conservation_rule)

def no_depot_to_depot_rule(model, v, i, j):
    if i in model.D and j in model.D and i != j:
        return model.x[v,i,j] == 0
    else:
        return Constraint.Skip

model.no_depot_travel = Constraint(model.V, model.N, model.N, rule=no_depot_to_depot_rule)

def customer_visit_rule(model, i):
    if i in model.C:
        return sum(model.x[v, j, i] for v in model.V for j in model.N if j != i) == 1
    else:
        return Constraint.Skip

model.customer_visit = Constraint(model.N, rule=customer_visit_rule)

def vehicle_capacity_rule(model, v):
    return sum(demand[i] * sum(model.x[v, j, i] for j in model.N if j != i) for i in model.C) <= vehicle_capacity[v]

model.vehicle_capacity = Constraint(model.V, rule=vehicle_capacity_rule)

def depot_capacity_rule(model, v):
    return sum(demand[i] * sum(model.x[v, j, i] for j in model.N if j != i) for i in model.C) <= sum(model.y[v,d] * depot_capacity[d] for d in model.D)

model.depot_capacity = Constraint(model.V, rule=depot_capacity_rule)

def vehicle_range_rule(model, v):
    return sum(model.distance[i,j] * model.x[v,i,j] for i in model.N for j in model.N if i != j) <= vehicle_range[v]

model.vehicle_range = Constraint(model.V, rule=vehicle_range_rule)

def mtz_rule(model, v, i, j):
    if i != j:
        return model.u[v,i] - model.u[v,j] + vehicle_capacity[v] * model.x[v,i,j] <= vehicle_capacity[v] - demand[j]
    return Constraint.Skip

model.mtz = Constraint(model.V, model.C, model.C, rule=mtz_rule)

def mtz_bounds_rule(model, v, i):
    return (demand[i], model.u[v,i], vehicle_capacity[v])

model.mtz_bounds = Constraint(model.V, model.C, rule=mtz_bounds_rule)

def demand_satisfaction_rule(model, v, i):
    return model.u[v,i] >= demand[i] * sum(model.x[v, j, i] for j in model.N if j != i)

model.demand_satisfaction = Constraint(model.V, model.C, rule=demand_satisfaction_rule)

from amplpy import modules

#solver = SolverFactory('glpk')
# resolver
solver_name = "gurobi" 
solver = SolverFactory(solver_name+"nl", executable=modules.find(solver_name), solve_io="nl")
results = solver.solve(model)

print("\nRutas óptimas:")
for v in model.V:
    assigned_depot = [d for d in model.D if value(model.y[v,d]) > 0.5]
    print("Vehículo", v, "asignado a bodega(s):", assigned_depot)
    for i in model.N:
        for j in model.N:
            if i != j and value(model.x[v,i,j]) > 0.5:
                print(" de", i, "a", j)

Gurobi 12.0.1:
Rutas óptimas:
Vehículo V1 asignado a bodega(s): ['CD9']
 de CD9 a C8
 de C5 a CD9
 de C7 a C5
 de C8 a C7
Vehículo V2 asignado a bodega(s): ['CD11']
 de CD11 a C6
 de C6 a CD11
Vehículo V3 asignado a bodega(s): ['CD5']
 de CD5 a C4
 de C1 a CD5
 de C4 a C1
Vehículo V4 asignado a bodega(s): ['CD11']
 de CD11 a C9
 de C9 a CD11
Vehículo V5 asignado a bodega(s): ['CD9']
 de CD9 a C3
 de C3 a CD9
Vehículo V6 asignado a bodega(s): ['CD5']
 de CD5 a C2
 de C2 a CD5


In [ ]:
!pip install openrouteservice folium

In [ ]:
import folium
import openrouteservice
from openrouteservice import convert
from pyomo.environ import value
import itertools

# Paleta de colores (puedes expandirla si tienes más vehículos)
colores = ['red', 'blue', 'green', 'orange', 'purple', 'brown', 'black', 'pink', 'gray', 'cyan']
vehiculo_a_color = dict(zip(model.V, itertools.cycle(colores)))


client = openrouteservice.Client(key=API_KEY)

m = folium.Map(location=[4.65, -74.1], zoom_start=11)

coords= lugares
labels= list(model.N.data())

In [ ]:
for label, coord in zip(labels, coords):
    folium.Marker(location=(coord[1], coord[0]), popup=label).add_to(m)

# Diccionario de índice a coordenada
label_to_coord = dict(zip(labels, coords))

# Supón que tienes model.x[v, i, j] y model.V y model.N cargado
# Simulación: rutas como lista de tuplas (vehículo, origen, destino)
rutas_optimas = []
for v in model.V:
    for i in model.N:
        for j in model.N:
            if i != j and value(model.x[v,i,j]) > 0.5:
                rutas_optimas.append((v, i, j))
print(rutas_optimas)



[('V1', 'CD9', 'C8'), ('V1', 'C5', 'CD9'), ('V1', 'C7', 'C5'), ('V1', 'C8', 'C7'), ('V2', 'CD11', 'C6'), ('V2', 'C6', 'CD11'), ('V3', 'CD5', 'C4'), ('V3', 'C1', 'CD5'), ('V3', 'C4', 'C1'), ('V4', 'CD11', 'C9'), ('V4', 'C9', 'CD11'), ('V5', 'CD9', 'C3'), ('V5', 'C3', 'CD9'), ('V6', 'CD5', 'C2'), ('V6', 'C2', 'CD5')]


In [ ]:
for (v, i, j) in rutas_optimas:
    coord_i = label_to_coord[i]
    coord_j = label_to_coord[j]
    color = vehiculo_a_color[v]
    try:
        route = client.directions(
            coordinates=[coord_i, coord_j],
            profile='driving-car',
            format='geojson'
        )
        folium.GeoJson(
            route,
            name=f"Ruta {v}: {i}->{j}",
            tooltip=f"{v}: {i} → {j}",
            style_function=lambda x, col=color: {'color': col, 'weight': 4, 'opacity': 0.8}
        ).add_to(m)
    except Exception as e:
        print(f"Error en ruta {i} → {j}: {e}")
m